In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,accuracy_score,f1_score,precision_score,roc_curve,auc 

%matplotlib inline

In [ ]:
#load dataset
dataset=ImageFolder('./dataset',transform=ToTensor())

#prepare for kfold
x=[]
y=[]
for i in dataset:
    img, label=i
    x.append(img.tolist())
    y.append(label)
x=np.array(x)
y=np.array(y)

#initialize parms
fold=1 #current fold
train_accuracy=[]
test_accuracy=[]
precision=[]
recall=[]
f1=[]
epoch=10000
learning=0.001
foldd=3 #total fold

#kfold
kfc = KFold(n_splits=foldd, random_state=1, shuffle=True)

#start
for train_index,test_index in kfc.split(x):
    #train test spilt
    xtrain=x[train_index]
    xtest=x[test_index]
    ytrain=y[train_index]
    ytest=y[test_index]
    ytrain=ytrain.reshape(1,ytrain.shape[0])
    ytest=ytest.reshape(1,ytest.shape[0])
    xtrain=xtrain.reshape(xtrain.shape[0],-1).T
    xtest=xtest.reshape(xtest.shape[0],-1).T
    
    print("Fold "+str(fold)+": ")
    
    #run odel
    test_acc,prec,rec,f=model(xtrain,ytrain,xtest,ytest,epoch,learning)
    
    #save result
    test_accuracy.append(test_acc)
    precision.append(prec)
    recall.append(rec)
    f1.append(f)
    fold+=1

#show result
print("Logicstic Regression Result "+str(epoch)+" epoch  Learning rate: "+str(learning)+" (mean of "+str(foldd)+" folds)")
print("Accuracy: ",sum(test_accuracy)/len(test_accuracy))
print("Precision: ",sum(precision)/len(precision))
print("F1: ",sum(f1)/len(f1))
print("Recall: ",sum(recall)/len(recall))

In [ ]:
#Logistic Regression

#sigmoid function
def sigmoid(z):
    a=1/(1+np.exp(-z))
    return a

#initialize
def initialize(dim):
    w=np.zeros((dim,1))
    b=0
    return w,b

#propagate(grade function) X from train data, y from train label
def propagate(w, b, X, Y):
    z=np.dot(w.T,X)+b
    a=sigmoid(z)
    m=X.shape[1]
    J=np.sum(((-np.log(a))*Y)+((-np.log(1-a))*(1-Y)))/m
    dw=np.dot(X,(a-Y).T)/m
    db=np.sum(a-Y)/m
    return J,dw,db

#gradient descent
def optimize(w,b,X,Y,iters,alpha):
    costs=[]
    ws=[]
    bs=[]
    for i in range (iters):
        cost,dw,db=propagate(w,b,X,Y)
        w=w-alpha*dw
        b=b-alpha*db
        if i%100==0:
            costs.append(cost)
        ws.append(w)
        bs.append(b)
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(alpha))
    plt.show()
    return w,b,dw,db,costs

#predict
def predict(w,b,X):
    m=X.shape[1]
    Y_predict=np.zeros((1,m))
    A=sigmoid(np.dot(w.T,X)+b)
    Y_predict=(A>=0.5)
    return Y_predict


def model(X_train,Y_train,X_test,Y_test,iters,alpha):
    w,b=initialize(X_train.shape[0])
    w,b,dw,db,costs=optimize(w,b,X_train,Y_train,iters,alpha)
    Y_train_predict=predict(w,b,X_train)
    Y_test_predict=predict(w,b,X_test)
    train_miss=0
    test_miss=0
    
    #caculate accuracy
    for i in range(X_train.shape[1]):
        if Y_train_predict[0][i]!=Y_train[0][i]:
            train_miss+=1
    train_acc=((X_train.shape[1]-train_miss)/X_train.shape[1])*100
    
    for i in range(X_test.shape[1]):
        if Y_test_predict[0][i]!=Y_test[0][i]:
            test_miss+=1
    test_acc=((X_test.shape[1]-test_miss)/X_test.shape[1])*100
    
    #plot and save result
    print("train accuracy:"+str(train_acc)+"%")
    print("test accuracy:"+str(test_acc)+"%")
    precision=precision_score(Y_test[0],Y_test_predict[0])
    recall=recall_score(Y_test[0],Y_test_predict[0])
    f1=f1_score(Y_test[0],Y_test_predict[0])
    print("Confusion matrix: ")
    print(confusion_matrix(Y_test[0],Y_test_predict[0]))
    fpr, tpr, threshold = roc_curve(Y_test[0],Y_test_predict[0])

    auc1 = auc(fpr, tpr)
    plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc1)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.show()
    print()
    return test_acc,precision,recall,f1